In [37]:
import pandas as pd
import re
import numpy as np
import nltk as nl
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.tag import pos_tag
import pickle
from collections import defaultdict
from collections import Counter
import math
import tqdm

In [52]:
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')

In [53]:
fake['label'] = 1
true['label'] = 0

In [54]:
fake.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [55]:
df = pd.concat([fake,true],ignore_index=True)

In [56]:
df

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1
...,...,...,...,...,...
44893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",0
44894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",0
44895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",0
44896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",0


In [57]:
df.shape

(44898, 5)

In [96]:
fake.subject.unique()

array(['News', 'politics', 'Government News', 'left-news', 'US_News',
       'Middle-east'], dtype=object)

In [97]:
true.subject.unique()

array(['politicsNews', 'worldnews'], dtype=object)

In [58]:
df.drop('subject', axis = 'columns',inplace=True) 
df.drop('date', axis = 'columns',inplace=True) 

In [59]:
# Modificare? Troppo simile indiano
df['text'] = df['title'] + ' ' + df['text'] 

In [60]:
df.drop('title', axis = 'columns',inplace=True) 

In [61]:
df.head()

,text,label
0,Donald Trump Sends Out Embarrassing New Year’...,1
1,Drunk Bragging Trump Staffer Started Russian ...,1
2,Sheriff David Clarke Becomes An Internet Joke...,1
3,Trump Is So Obsessed He Even Has Obama’s Name...,1
4,Pope Francis Just Called Out Donald Trump Dur...,1


# Preprocessing

In [49]:
def preprocess(data):
    '''
    tagged = pos_tag(data.split())
    names = [t[0] for t in tagged if t[1] == 'NNP']
    print(names)
    '''
    x = re.sub('[^a-zA-Z]', ' ',data) 
    
    #lowering words
    lower=str.lower(x).split() 
    words=set(stopwords.words('english'))
    
    #removing stopwords
    no_stopwords=[w for w in lower if not w in words]  
    lmtzr = WordNetLemmatizer()
    
    #stemming
    cleaned=[lmtzr.lemmatize(w) for w in no_stopwords]
    print(no_stopwords)
    
    return (" ".join( cleaned ))

In [65]:
def preprocess2(data):
    '''
    tagged = pos_tag(data.split())
    names = [t[0] for t in tagged if t[1] == 'NNP']
    print(names)
    '''
    x = re.sub('[^a-zA-Z]', ' ',data) 
    
    #lowering words
    lower=str.lower(x).split() 
    words=set(stopwords.words('english'))
    
    #removing stopwords
    no_stopwords=[w for w in lower if not w in words]  
    lmtzr = WordNetLemmatizer()
    
    #stemming
    cleaned=[lmtzr.lemmatize(w,pos="v") for w in no_stopwords]
    cleaned=[lmtzr.lemmatize(w,pos="n") for w in cleaned]
    
    return (" ".join( cleaned ))

In [50]:
preprocess(df.iloc[0]['text'])

['donald', 'trump', 'sends', 'embarrassing', 'new', 'year', 'eve', 'message', 'disturbing', 'donald', 'trump', 'wish', 'americans', 'happy', 'new', 'year', 'leave', 'instead', 'give', 'shout', 'enemies', 'haters', 'dishonest', 'fake', 'news', 'media', 'former', 'reality', 'show', 'star', 'one', 'job', 'country', 'rapidly', 'grows', 'stronger', 'smarter', 'want', 'wish', 'friends', 'supporters', 'enemies', 'haters', 'even', 'dishonest', 'fake', 'news', 'media', 'happy', 'healthy', 'new', 'year', 'president', 'angry', 'pants', 'tweeted', 'great', 'year', 'america', 'country', 'rapidly', 'grows', 'stronger', 'smarter', 'want', 'wish', 'friends', 'supporters', 'enemies', 'haters', 'even', 'dishonest', 'fake', 'news', 'media', 'happy', 'healthy', 'new', 'year', 'great', 'year', 'america', 'donald', 'j', 'trump', 'realdonaldtrump', 'december', 'trump', 'tweet', 'went', 'welll', 'expect', 'kind', 'president', 'sends', 'new', 'year', 'greeting', 'like', 'despicable', 'petty', 'infantile', 'gib

'donald trump sends embarrassing new year eve message disturbing donald trump wish american happy new year leave instead give shout enemy hater dishonest fake news medium former reality show star one job country rapidly grows stronger smarter want wish friend supporter enemy hater even dishonest fake news medium happy healthy new year president angry pant tweeted great year america country rapidly grows stronger smarter want wish friend supporter enemy hater even dishonest fake news medium happy healthy new year great year america donald j trump realdonaldtrump december trump tweet went welll expect kind president sends new year greeting like despicable petty infantile gibberish trump lack decency even allow rise gutter long enough wish american citizen happy new year bishop talbert swan talbertswan december one like calvin calvinstowell december impeachment would make great year america also accept regaining control congress miranda yaver mirandayaver december hear talk include many p

In [48]:
preprocess2(df.iloc[0]['text'])

['donald', 'trump', 'sends', 'embarrassing', 'new', 'year', 'eve', 'message', 'disturbing', 'donald', 'trump', 'wish', 'americans', 'happy', 'new', 'year', 'leave', 'instead', 'give', 'shout', 'enemies', 'haters', 'dishonest', 'fake', 'news', 'media', 'former', 'reality', 'show', 'star', 'one', 'job', 'country', 'rapidly', 'grows', 'stronger', 'smarter', 'want', 'wish', 'friends', 'supporters', 'enemies', 'haters', 'even', 'dishonest', 'fake', 'news', 'media', 'happy', 'healthy', 'new', 'year', 'president', 'angry', 'pants', 'tweeted', 'great', 'year', 'america', 'country', 'rapidly', 'grows', 'stronger', 'smarter', 'want', 'wish', 'friends', 'supporters', 'enemies', 'haters', 'even', 'dishonest', 'fake', 'news', 'media', 'happy', 'healthy', 'new', 'year', 'great', 'year', 'america', 'donald', 'j', 'trump', 'realdonaldtrump', 'december', 'trump', 'tweet', 'went', 'welll', 'expect', 'kind', 'president', 'sends', 'new', 'year', 'greeting', 'like', 'despicable', 'petty', 'infantile', 'gib

'donald trump send embarrass new year eve message disturb donald trump wish american happy new year leave instead give shout enemy hater dishonest fake news medium former reality show star one job country rapidly grow stronger smarter want wish friend supporter enemy hater even dishonest fake news medium happy healthy new year president angry pant tweet great year america country rapidly grow stronger smarter want wish friend supporter enemy hater even dishonest fake news medium happy healthy new year great year america donald j trump realdonaldtrump december trump tweet go welll expect kind president send new year greet like despicable petty infantile gibberish trump lack decency even allow rise gutter long enough wish american citizen happy new year bishop talbert swan talbertswan december one like calvin calvinstowell december impeachment would make great year america also accept regain control congress miranda yaver mirandayaver december hear talk include many people hate wonder ha

In [67]:
df['text'] = df.text.apply(lambda x: preprocess2(x))

In [68]:
df.head()

,text,label
0,donald trump send embarrass new year eve messa...,1
1,drink brag trump staffer start russian collusi...,1
2,sheriff david clarke become internet joke thre...,1
3,trump ob even obama name cod website image chr...,1
4,pope francis call donald trump christmas speec...,1


In [69]:
df.to_csv('preprocessed.csv')

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    44898 non-null  object
 1   label   44898 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 701.7+ KB


In [3]:
df = pd.read_csv('/Users/domenicomattiacinque/Documents/Università/FDS2020/preprocessed.csv')

In [4]:
#saving dict function
def save_dict(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

#load dict function
def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [5]:
def create_vocabulary(df):

    #the Dict we wanna build
    vocabulary = defaultdict()

    #set in which i collect all the terms
    term_set = set()
    text = list(df['text'])
    for elem in text:
        try:
            term_set =term_set.union(set(elem.split()))
        except:
            pass

    #convert the set in list to enumerate
    term_list = list(term_set)

    for i, elem in enumerate(term_list):
        vocabulary[elem]= i 

    save_dict(vocabulary,'vocabulary')

In [6]:
create_vocabulary(df)

KeyboardInterrupt: 

In [74]:
vocabulary = load_obj('vocabulary')

In [75]:
len(vocabulary)

98132

In [15]:
def mapping(text):
    text = text.split()
    out = [vocabulary[word] for word in text]
    return out

In [16]:
df['mapped_text'] = df.text.apply(lambda x: mapping(x))

In [17]:
df.head()

,Unnamed: 0,text,label,mapped_text
0,0,donald trump send embarrass new year eve messa...,1,"[70224, 48026, 71767, 29317, 7203, 7759, 73615..."
1,1,drink brag trump staffer start russian collusi...,1,"[9047, 46709, 48026, 46078, 34043, 20147, 2169..."
2,2,sheriff david clarke become internet joke thre...,1,"[30383, 91062, 77676, 23464, 66433, 8578, 8769..."
3,3,trump ob even obama name cod website image chr...,1,"[48026, 82760, 94341, 64354, 72398, 5206, 6520..."
4,4,pope francis call donald trump christmas speec...,1,"[26459, 24198, 73700, 70224, 48026, 62043, 834..."


In [18]:
df.to_csv('mapped_dataset.csv')

# Parameters

In [59]:
train = df.sample(n = math.floor(0.8*len(df)))
train.shape

(35918, 4)

In [60]:
test = df.drop(train.index)

In [61]:
test.shape

(8980, 4)

In [56]:
train.to_csv('train.csv')
test.to_csv('test.csv')

In [62]:
train.head()

,Unnamed: 0,text,label,mapped_text
25083,25083,mediation end minnesota budget dispute fizzle ...,0,"[81570, 69176, 19918, 68693, 43509, 94309, 318..."
30419,30419,taiwan call china view trump call calmly taipe...,0,"[38317, 73700, 39132, 43614, 48026, 73700, 250..."
41504,41504,speaker ethiopian parliament submit resignatio...,0,"[46893, 50875, 66045, 66972, 88202, 23040, 567..."
14972,14972,wow ky dem house speaker make insane speech fo...,1,"[11208, 23869, 26190, 66672, 46893, 43504, 964..."
14628,14628,heartless marxist bernie sander homeless peopl...,1,"[23701, 98110, 71702, 1880, 26104, 13986, 4043..."


In [67]:
train.reset_index(inplace=True)

In [68]:
train = train.rename(columns = {'index':'id'})

In [69]:
train.head()

,id,Unnamed: 0,text,label,mapped_text
0,25083,25083,mediation end minnesota budget dispute fizzle ...,0,"[81570, 69176, 19918, 68693, 43509, 94309, 318..."
1,30419,30419,taiwan call china view trump call calmly taipe...,0,"[38317, 73700, 39132, 43614, 48026, 73700, 250..."
2,41504,41504,speaker ethiopian parliament submit resignatio...,0,"[46893, 50875, 66045, 66972, 88202, 23040, 567..."
3,14972,14972,wow ky dem house speaker make insane speech fo...,1,"[11208, 23869, 26190, 66672, 46893, 43504, 964..."
4,14628,14628,heartless marxist bernie sander homeless peopl...,1,"[23701, 98110, 71702, 1880, 26104, 13986, 4043..."


In [32]:
n = len(train)
ID = defaultdict(list)

for i,row in train.iterrows():
    p = set(row['text'].split())
    for word in p:
        ID[vocabulary[word]].append(i)    

with open('ID.pkl', 'wb') as f:
    pickle.dump(ID, f, pickle.HIGHEST_PROTOCOL)
    f.close()

In [40]:
def frequency_of_words_per_article(df):      
    
    frequency_of_word = []

    #each dictionary has for keys all the words in the dictionary file we created above
    df1 = df['text']

    for i in range(len(df)):
        row = df1.loc[i].split()
        frequency_of_words = Counter(row)
        frequency_of_word.append(frequency_of_words)
    
    return frequency_of_word

In [72]:
def inverted_index1(df,dictionary,frequency_of_words):  
    
    inverted_index1 = dict.fromkeys(dictionary.values(),[])
    for key, value in dictionary.items():
        elem = []
        for i in range(len(df)):
            if frequency_of_words[i][key] > 0:
                elem.append((df.iloc[i]['id'], frequency_of_words[i][key]))
        inverted_index1[value] = elem 
    
    return inverted_index1

In [ ]:
inverted_index = inverted_index1(train, vocabulary, frequency_of_words_per_article(train))

In [24]:
phi_y = train['label'].sum()/len(train)

In [25]:
phi_y

0.5213541956679102

In [26]:
n = len(vocabulary)

#0 real 1 fake
phi_0 = np.zeros(n) 
phi_1 = np.zeros(n)

d = len(vocabulary)
d_0 = sum([len(i) for i in train_0.mapped_text])
norm_0 = len(train_0)
norm_1 = len(train_1)

for i in range(n):
    phi_0[i] = 
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-26-89f7d48c532f>, line 8)